ą# Prezentacja o bazie NoSQL Qdrant

---

## Autorzy

- Mateusz Malich, Aleksander Batko

---

## Bibliografia

- Oficjalna strona Qdrant: [https://qdrant.tech](https://qdrant.tech)  
- Repozytorium GitHub Qdrant: [https://github.com/qdrant/qdrant](https://github.com/qdrant/qdrant)  
- Artykuły i blogi techniczne o Qdrant (np. Medium, Towards Data Science)  
- Dokumentacja Qdrant: [https://qdrant.tech/documentation/](https://qdrant.tech/documentation/)
- Baza wektorowa: [https://learn.microsoft.com/pl-pl/fabric/real-time-intelligence/vector-database](https://learn.microsoft.com/pl-pl/fabric/real-time-intelligence/vector-database)

---

## Podstawowe informacje o bazie i jej historii

**Qdrant** to nowoczesna baza danych NoSQL typu **vector database**, zaprojektowana do przechowywania i szybkiego wyszukiwania wektorów osadzania (embeddings) generowanych przez modele AI.

### Cechy Qdrant:
- Wysoka wydajność wyszukiwania milionów wektorów przy niskim opóźnieniu  
- Obsługa metadanych do zaawansowanego filtrowania wyników  
- Prosty REST API i gRPC do integracji  
- Możliwość tworzenia i usuwania kolekcji w czasie działania  
- Open Source na licencji Apache 2.0  
- Dostępna lokalnie (on-premises) oraz jako chmurowa usługa Qdrant Cloud  

### Historia:
- Projekt rozpoczęty w 2021 roku przez Qdrant GmbH z Berlina  
- Publiczne wydanie open source w 2022  
- Dynamiczny rozwój w latach 2023-2024 z naciskiem na wydajność i łatwość użycia  

---

### Czym jest baza wektorowa?
Baza wektorowa to specjalny rodzaj bazy danych zaprojektowany do przechowywania i szybkiego wyszukiwania danych w postaci wektorów liczbowych (np. list liczb). Wektory te często powstają jako reprezentacje (embeddingi) obiektów takich jak teksty, obrazy, dźwięki czy inne dane, które dzięki temu można porównywać pod kątem ich znaczenia lub podobieństwa.

## Dlaczego to ważne?
Tradycyjne bazy danych świetnie radzą sobie z danymi strukturalnymi i wyszukiwaniem po kluczach lub pełnotekstowym. Jednak gdy chcemy wyszukiwać np. „podobne obrazy” lub „teksty o podobnym sensie”, potrzebujemy porównywać dane w przestrzeni wielowymiarowej. Do tego służą bazy wektorowe.

### Czym jest embedding?
Embedding to sposób przedstawienia różnych obiektów (np. słów, zdań, obrazów) jako wektorów liczb w przestrzeni wielowymiarowej.
Dane takie jak tekst czy obraz są trudne do bezpośredniego przetwarzania przez komputer na poziomie matematycznym. Embedding pozwala przekształcić te dane do postaci ciągu liczb (wektora), które zachowują ich znaczenie lub cechy.

Na przykład:
- Słowa o podobnym znaczeniu będą miały podobne wektory (blisko siebie w przestrzeni).
- Obrazy podobnych przedmiotów będą miały zbliżone reprezentacje.

### Jak tworzy się wektory?
W zalezności od rodzaju danych wejściowych (tekst, obraz, dzwięk) to zazwyczaj model uczenia maszynowego zamienia dane wejściowe na wektor.
W przypadku tekstu model językowy przetwarza słowa na reprezentacje wektorowe, biorąc pod uwagę kontekst, znaczenie i relacje między słowami.
Przy obrazie sieć konwolucyjna analizuje obraz i wyciąga cechy, tworząc wektor cech.
Jezeli plik wejściowy to dzwięk, model analizuje cechy akustyczne i konwertuje je na wektor

Model zwraca wektor o stałej długości, np. 128 czy 768 wymiarów, który jest gęstą reprezentacją danych.
Ten wektor zachowuje istotne cechy obiektu — np. podobne zdania mają podobne wektory.

### Konkurencja dla Qdranta
- PostgreSQL mozna rozszerzyć o plugin "pgvektor"
- Pinecone
- Weaviate
- Faiss (Facebook AI Similarity Search)

## Praktyczny przykład
- Obiekt (np. zdanie) jest zamieniany na wektor liczb (embedding) za pomocą modelu AI
- Wektory umieszczane są w bazie
- Podczas wyszukiwania podajesz zapytanie, które również jest przekształcane na wektor
- Baza szybko znajduje wektory najbardziej „podobne” do zapytania, np. na podstawie odległości kosinusowej lub euklidesowej
---

### Firmy korzystające z Qdrant:

- Microsoft
- Disney dla Disney+
- Kaufland 
- X (Twitter)
- Discord 

## Instalacja i dostępność

### Instalacja lokalna

Na początku trzeba pobrać najnowszy obraz Qdrant z Dockerhuba:

```bash
docker pull qdrant/qdrant
```bash

Potem uruchom jako usługe:

```bash
docker run -p 6333:6333 -p 6334:6334 \
    -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant
```bash